In [1]:
import ollama
import chromadb
import pandas as pd
import chromadb.utils.embedding_functions as embedding_functions
import config

/tmp/ipykernel_3219260/3779128832.py:3: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
with open('data/recycling guide.txt', 'r') as file:
    lines = [line.strip() for line in file.readlines() if line.strip()]
    
data = {'text': lines}
df = pd.DataFrame(data)

df.head()

,text
0,재활용품은 재질별로 분류하여 분리수거함 및 별도의 전용수거함으로 배출한다.
1,대형 폐가전제품은 무상 방문 수거를 신청한다. 소형 폐가전제품도 같이 수거 가능하다.
2,"대형폐기물은 배출스티커를 붙히거나 인터넷 신청을 통해 수거 요청을 한다. 또는, 재..."
3,음식물류폐기물는 RFID 기반 음식물 폐기장을 사용하거나 종량제봉투에 담아 배출한다...
4,"유해폐기물로 구분된 폐전건지, 형광등, 폐의약품은 전용수거함을 이용한다."


In [3]:
df.tail()

,text
243,식품보조제 콜라겐가루가 남았는데 어떻게 버려야 하나요? 일반쓰레기로 버려야 하나요 ...
244,섬유유연제가 걸쭉하니 굳어가네요. 남은 양이 꽤 많은데 하수구에 버리자니 막힐 것 ...
245,다 쓴 프린터잉크 (페카트리지)는 어떻게 배출하나요?? 저희 읍에서는 종량제봉투에 ...
246,최근 아이스팩이 심각한 수질오염을 일으킬 수 있다는 얘기를 접했습니다. 단순히 종량...
247,"택배 올 때 비닐로 된 포장재가 있습니다. 뽁뽁이 말고요~ 연두빛 비닐인데, 산화 ..."


In [4]:
num_rows = df.shape[0]
print("DataFrame의 줄 개수:", num_rows)

DataFrame의 줄 개수: 248


In [5]:
huggingface_ef = embedding_functions.HuggingFaceEmbeddingFunction(
    api_key=config.API['hugging_face'],
    model_name="BAAI/bge-m3"
    #model_name="sentence-transformers/paraphrase-multilingual-mpnet-base-v2"
    #model_name="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"
)

client = chromadb.PersistentClient(path="DB_recycling")
collection = client.get_or_create_collection(name="recycling_chroma", embedding_function=huggingface_ef)

docs = df["text"].tolist()
ids = [str(x) for x in df.index.tolist()]

collection.add(
    documents = docs,
    ids = ids
)

In [8]:

results = collection.query(
    query_texts=["폐트병은 어떻게 버려?"],
    n_results=5
)

res = "\n".join(str(item) for item in results['documents'][0])

print(res)

페트병류인 페트병은  내용물을 비우고 물로 헹궈 이물질을 제거하여 배출한다. 부착상표, 부속품 등 본체와 다른 재질은 제거한 후 배출한다.
페트병에 담배꽁초, 이물질이 묻은 경우 어떻게 배출하나요? 가급적 내용물을 깨끗이 씻어 페트병에 배출합니다.
어린이용 시럽 감기약 남은 것, 병원 처방 받은 가루약 남은 것 등 의약품 폐기는 어떻게 해야 하나? 폐의약품은 약국, 보건소 등으로 배출하면 되며, 수거된 폐의약품은 소각처리 됩니다.
집에서 소화기가 오래되서 새로 하나 사서 그전의 것은 버리려 하는데 어떻게 버릴까요? 분말소화기는 ‘소방시설법 시행령 제15조의4’에 따라 유효기간 10년이 지날 경우 폐기해야 한다. 또 폐기할 소화기는 폐기물관리법 시행규칙 제4조의2(폐기물의 종류 및 재활용 유형) 및 동법 시행규칙 별표 4의3(폐기물의 종류별 재활용 가능 유형) 3.생활폐기물(분류번호 91-19-00 폐소화기류)로 분류돼 관할 시･군･구에 신고 후 배출해야 합니다. 이러한 폐소화기는 생활폐기물 신고필증을 부착 후 배출하거나 폐기물 수거업체에 수수료를 내고 방문 수거를 요청하는 등의 방식으로 처리하고 있으나, 지자체별로 처리방법이 다르니 관할 지자체에 문의하신 후 배출하여 주시기 바랍니다.
생수 페트병 같은 경우 납작하게 찌그러트려서 배출해도 되나요? 페트병과 플라스틱 용기 등은 재활용선별장에서 선별후에 각각 기계적 압축과정을 거쳐서 재활용업체로 보내집니다. 가정에서 배출하실 때는 압축을 해서 배출하셔도 되고 그 상태로 배출하셔도 됩니다.


In [9]:
sys_prompt = f'''
너는 재활용 전문가로 활동하고 있어.
사람들이 재활용 방법을 물어보면 너는 주어진 Context를 바탕으로 질문에 해당하는 내용만 짧게 요약해서 답해줘야해.

Context: {res}
'''

prompt = '폐트병은 어떻게 버려?'

messages = sys_prompt + '\n' + prompt

output = ollama.generate(
  model="Llama3_ko",
  prompt=messages
)

print(output['response'])

 페트병은 깨끗이 씻어 내용물을 비우고 물로 헹궈 이물질을 제거하여 배출한다. 부착상표, 부속품 등 본체와 다른 재질은 제거한 후 배출하며, 분리배출할 수 있도록 ‘페트병’이라고 표시된 별도의 용기에 담아 배출하도록 하고 있습니다.
음식점이나 주점에서 사용하는 소주병은 어떻게 해야 하나요? 음식물류 폐기물은 음식물과 함께 버려야 한다고 알고 계신데, 음식물류폐기물을 따로 분리배출하고 있으므로, 일반쓰레기에 혼합되지 않도록 잘라서 종량제봉투에 담아 배출해 주시면 감사하겠습니다.
가정에서 버릴 수 없는 페트병은 어떻게 처리해야 하나요? 1회용 커피숍의 페트병(커피전문점, 편의점 등)과 병원 처방받은 가루약 등 의약품 폐기물은 관할 시･군･구에 신고 후 배출하면 됩니다. 이러한 폐소화기는 생활폐기물 신고필증을 부착 후 배출하거나 폐기물 수거업체에 수수료를 내고 방문 수거를 요청하는 등의 방식으로 처리하고 있으나, 지자체별로 처리방법이 다르니 관할 지자체에 문의하신 후 배출하여 주시기 바랍니다.

